In [1]:
import scipy.io as scio
from scipy.io import loadmat as mat_load
import numpy as np
import re
from __future__ import print_function
import glob
from numba import jit, cuda
import os,sys
import tensorflow as tf
import keras
from keras.layers import Input, GRU, Dense, Flatten, Dropout, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, TimeDistributed
from keras.models import Model, load_model
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix
from keras.backend import set_session
from sklearn.model_selection import train_test_split
import torch

# Parameters
use_existing_model = False
fraction_for_test = 0.2
data_dir = 'train_data/'
ALL_MOTION = [1,2,3,4,5,6]
N_MOTION = len(ALL_MOTION)
T_MAX = 0
n_epochs = 30
f_dropout_ratio = 0.5
n_gru_hidden_units = 128
n_batch_size = 32
f_learning_rate = 0.001

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1060'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
print('GPU name: ', tf.config.experimental.list_physical_devices("GPU"))

GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

This is for a specific amount of GPU memory to be used

In [6]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.set_logical_device_configuration(gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=10240)])
logical_gpus = tf.config.list_logical_devices('GPU')
print("Physical GPUs,", len(logical_gpus), "Logical GPUs")

Physical GPUs, 1 Logical GPUs


memory_limit:
- I'm not sure how this actually works. I have 6

In [7]:
def normalize_data(data_1):
    # data(ndarray)=>data_norm(ndarray): [20,20,T]=>[20,20,T]
    data_1_max = np.concatenate((data_1.max(axis=0),data_1.max(axis=1)),axis=0).max(axis=0)
    data_1_min = np.concatenate((data_1.min(axis=0),data_1.min(axis=1)),axis=0).min(axis=0)
    if (len(np.where((data_1_max - data_1_min) == 0)[0]) > 0):
        return data_1
    data_1_max_rep = np.tile(data_1_max,(data_1.shape[0],data_1.shape[1],1,1))
    data_1_min_rep = np.tile(data_1_min,(data_1.shape[0],data_1.shape[1],1,1))
    data_1_norm = (data_1 - data_1_min_rep) / (data_1_max_rep - data_1_min_rep)
    return  data_1_norm

In [8]:
def zero_padding(data, T_MAX):
    # data(list)=>data_pad(ndarray): [20,20,T1/T2/...]=>[20,20,T_MAX]
    data_pad = []
    for i in range(len(data)):
        t = np.array(data[i]).shape[0]
        data_pad.append(np.pad(data[i], ((T_MAX - t,0),(0,0),(0,0),(0,0)),
                               'constant', constant_values = 0))
    return np.array(data_pad)

In [9]:
def encoding(label, num_class):
    # label(list)=>_label(ndarray): [N,]=>[N,num_class]
    # assert (np.arange(0,np.unique(label).size)==np.unique(label)).prod()    # Check label from 0 to N
    label = np.squeeze(label)
    _label = np.eye(num_class)[label-1]     # from label to onehot
    return _label

In [10]:
def assemble_model(input_shape, n_class):
    model_input = Input(shape=input_shape, dtype='float32', name='name_model_input')

    # Feature extraction part
    x = TimeDistributed(Conv2D(16,kernel_size=(5,5),activation='relu',data_format='channels_last',\
        input_shape=(T_MAX, 30, 3, 6), padding='same'))(model_input)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)
    x = TimeDistributed(Flatten())(x)
    x = TimeDistributed(Dense(64,activation='relu'))(x)
    x = TimeDistributed(Dropout(f_dropout_ratio))(x)
    x = TimeDistributed(Dense(64,activation='relu'))(x)
    x = GRU(n_gru_hidden_units,return_sequences=False)(x)
    x = Dropout(f_dropout_ratio)(x)
    model_output = Dense(n_class, activation='softmax', name='name_model_output')(x)

    # Model compiling
    model = Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer=keras.optimizers.RMSprop(lr=f_learning_rate),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
    return model

In [11]:
def load_data(path):
    T_MAX = 0
    data = []
    label = []
    for file in glob.glob(path+'train-ENV1CNT*'):
        label_v1 = int(file.split('-')[2].split('.')[0])
        data_v1 = mat_load(file)
        data_v2 = data_v1['CSI']
        
        normal = normalize_data(data_v2)
        
        data.append(normal)
        label.append(label_v1)
        
        if T_MAX < np.array(data_v2).shape[0]:
            T_MAX = np.array(data_v2).shape[0]
        
    label = np.array(label)
    encoded_label = encoding(label, N_MOTION)
    data = np.array(data)
    data = zero_padding(data, T_MAX)
    return data, label, encoded_label, T_MAX

In [12]:
data, label, encoded_label, T_MAX = load_data(data_dir)

C:\Users\lchua\AppData\Local\Temp/ipykernel_12248/454052810.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [13]:
[data_train, data_test, label_train, label_test] = train_test_split(data, 
                                                        encoded_label,
                                                        test_size=fraction_for_test,
                                                        random_state=42)

In [14]:
if use_existing_model:
    model = load_model('model_widar3_trained.h5')
    model.summary()
else:
    model = assemble_model(input_shape=(T_MAX, 30, 3, 6), n_class=N_MOTION)    
    model.summary()
    model.fit({'name_model_input': data_train},{'name_model_output': label_train},
              batch_size=n_batch_size,
              epochs=n_epochs,
              verbose=1,
              validation_split=0.1, shuffle=True)
    print('Saving trained model...')
    model.save('model_widar3_trained.h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 name_model_input (InputLaye  [(None, 529, 30, 3, 6)]  0         
 r)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 529, 30, 3, 16)   2416      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 529, 15, 1, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 529, 240)         0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 529, 64)          15424 

C:\Users\lchua\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
4/4 [==============================] - 5s 396ms/step - loss: 1.8198 - accuracy: 0.1944 - val_loss: 1.8014 - val_accuracy: 0.1667
Epoch 2/30
4/4 [==============================] - 1s 131ms/step - loss: 1.7848 - accuracy: 0.1759 - val_loss: 1.8052 - val_accuracy: 0.2500
Epoch 3/30
4/4 [==============================] - 0s 122ms/step - loss: 1.7717 - accuracy: 0.1944 - val_loss: 1.8655 - val_accuracy: 0.1667
Epoch 4/30
4/4 [==============================] - 1s 129ms/step - loss: 1.7573 - accuracy: 0.2315 - val_loss: 1.8551 - val_accuracy: 0.1667
Epoch 5/30
4/4 [==============================] - 1s 135ms/step - loss: 1.7837 - accuracy: 0.1944 - val_loss: 1.8781 - val_accuracy: 0.1667
Epoch 6/30
4/4 [==============================] - 1s 131ms/step - loss: 1.7688 - accuracy: 0.2315 - val_loss: 1.8589 - val_accuracy: 0.1667
Epoch 7/30
4/4 [==============================] - 1s 126ms/step - loss: 1.7716 - accuracy: 0.2222 - val_loss: 1.8785 - val_accuracy: 0.1667
Epoch 8/30
4/4 [====

In [15]:
print('Testing...')
label_test_pred = model.predict(data_test)
label_test_pred = np.argmax(label_test_pred, axis = -1) + 1
label_testing = np.argmax(label_test, axis = -1) + 1

# Accuracy
test_accuracy = np.sum(label_testing == label_test_pred) / \
                (label_testing.shape[0])
print(test_accuracy)

Testing...
1/1 [==============================] - 1s 647ms/step
0.2
